In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, Sequential, load_model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/StressAnalysis")

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

vocab_file = 'vocab.txt'
class_names = ['normal', "depressed"]

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped = tf.strings.regex_replace(lowercase, "\n", " ")
    return tf.strings.regex_replace(
        stripped, "[%s]" % re.escape(string.punctuation), ""
    )

# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 1000

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

print ("attempting to recover previous vocabulary...")
f = open(vocab_file, 'r')
vocab = f.read().split('\n')
vocab = vocab[1:]
vectorize_layer.set_vocabulary(vocab, df_data=None, oov_df_value=None)
print ('previous vocabulary loaded')


attempting to recover previous vocabulary...
previous vocabulary loaded


In [ ]:
model = load_model('binary_model_LSTM.h5')
model.summary()
model.compile()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
dropout_9 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256

In [ ]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [ ]:
#predict function
def predict_post(string):
    pred = end_to_end_model.predict(string)
    print(pred)
    pred = int(np.round(pred[0][0]))
    print (pred)
    print (class_names[pred])

In [ ]:
text = ['I am tired \n tired of waiting, I am not waiting anymore I want it to be over it hurts me and my family.']
predict_post(text)

[[1.4467659e-07]]
0
normal


In [ ]:
text = input()
predict_post([text])